In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

#Import Parser
from langchain_core.output_parsers import (
    CommaSeparatedListOutputParser,
    JsonOutputParser,
    PydanticOutputParser
)
from pydantic import BaseModel, Field

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")
    exit(1)

In [ ]:
# Initialize the OpenAI model
llm = ChatOpenAI(
    openai_api_key=api_key,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1000
)

### Example 1: CommaSeparatedListOutputParser

In [ ]:
print("Example 1: CommaSeparatedListOutputParser")

# Create parser instance
list_parser = CommaSeparatedListOutputParser()

# Get format instructions (optional but helpful for LLM)
format_instructions = list_parser.get_format_instructions()
print(f"Format Instructions: {format_instructions}")

# Create a prompt asking for a list and including instructions
list_prompt_template = ChatPromptTemplate.from_template(
    "List 5 major Machine Learning tools. {format_instructions}"
)

# Format the prompt
list_prompt = list_prompt_template.format(format_instructions=format_instructions)
print(f"Prompt: {list_prompt}") # See how instructions are included

# Call the LLM with the prompt
response = llm.invoke(list_prompt)

# Parse the response
try:
    parsed_list = list_parser.parse(response.content)
    print(f"Parsed List: {parsed_list}")
    print(f"Parsed List Type: {type(parsed_list)}")
    print(f"First Item: {parsed_list[0]}")
except Exception as e:
    print(f"Error parsing response: {e}")
    print(f"Raw Response: {response.content}")
    exit(1)


### Example 2: PydanticOutputParser

In [ ]:
print("Example 2: PydanticOutputParser")

# Define desired output structure using Pydantic
class PersonInfo(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    city: str = Field(description="City where the person lives")

# Create pydantic parser instance
pydantic_parser = PydanticOutputParser(pydantic_object=PersonInfo)

# Get format instructions (these are crucial for the LLM to understand the expected output.)
format_instructions_pydantic = pydantic_parser.get_format_instructions()
print(f"Format Instructions: {format_instructions_pydantic}")

# Create prompt to extract information and include instructions
pydantic_prompt_template = ChatPromptTemplate.from_template(
    "Extract the name, age, and city of a person. {format_instructions}\n"
    "Text:{text_input}"
)

# 5. Input text containing the information
text = "Alice is 30 years old and resides in New York City."

# Format the prompt
pydantic_prompt = pydantic_prompt_template.format(
    format_instructions=format_instructions_pydantic,
    text_input=text
)
print(f"Prompt: {pydantic_prompt}") # See how instructions are included

# Call the LLM with the prompt
response = llm.invoke(pydantic_prompt)
# Parse the response
try:
    parsed_info = pydantic_parser.parse(response.content)
    print(f"Parsed Info: {parsed_info}")
    print(f"Parsed Info Type: {type(parsed_info)}")
    print(f"Name: {parsed_info.name}")
    print(f"Age: {parsed_info.age}")
    print(f"City: {parsed_info.city}")
except Exception as e:
    print(f"Error parsing response: {e}")
    print(f"Raw Response: {response.content}")
    exit(1)

### Example 3: JSON Output Parser

In [ ]:
# Create a json parser instance
# JSonOutputParser is a generic parser that can be used to parse JSON strings into Python dictionaries.
# JSsonOutputParser can optionally take Pydantic model as an argument to validate the parsed JSON data.
# If a Pydantic model is provided, the parser will validate the parsed JSON data against the model.

json_parser = JsonOutputParser()

# Define the desired JSON structure in the PROMPT instuction.
# Note: Using triple quotes for multi-line f-strings
json_prompt_template_text = """
    Return JSON object with the following keys:
    'company_name': The name of the company
    'ticker_symbol': The stock ticker symbol
    'industry': The industry the company operates in
    'market_cap': The market capitalization of the company
    'ceo': The name of the CEO
    'headquarters': The location of the company's headquarters

    Provide the information for the following company: {company_query}

    Format Instruction: Ensure the output is a valid JSON object. Do not incude text before or after the JSON object. for example:
    {{
        "company_name": "Apple Inc.",
        "ticker_symbol": "AAPL",
        "industry": "Technology",
        "market_cap": 2.5e12,
        "ceo": "Tim Cook",
        "headquarters": "Cupertino, California" 
    }}
"""

In [ ]:
# Create prompt template
json_prompt_template = ChatPromptTemplate.from_template(
    json_prompt_template_text
)
# Format the prompt
company_query = "Tesla Inc."
json_prompt = json_prompt_template.format(company_query=company_query)

response = llm.invoke(json_prompt)

In [ ]:
try:
    parsed_json = json_parser.parse(response.content)
    print(f"Parsed JSON: {parsed_json}")
    print(f"Parsed JSON Type: {type(parsed_json)}")
    print(f"Company Name: {parsed_json['company_name']}")
    print(f"Ticker Symbol: {parsed_json['ticker_symbol']}")
    print(f"Industry: {parsed_json['industry']}")
    print(f"Market Cap: {parsed_json['market_cap']}")
    print(f"CEO: {parsed_json['ceo']}")
    print(f"Headquarters: {parsed_json['headquarters']}")
except Exception as e:
    print(f"Error parsing response: {e}")
    print(f"Raw Response: {response.content}")
    exit(1)